In [2]:
import pandas as pd
import numpy as np
import config
from sqlalchemy import text
print(config.version)
print(config.connection_string)
pd.set_option('display.max_columns', None)

3.2.0.Dev
postgresql://postgres:sa@localhost/icip


In [3]:
damir = pd.read_sql("select * from apl.damir_age_stats where psp_spe_snds > 0 and psp_spe_snds <= 42", config.connection_string)
damir

,psp_spe_snds,ann,age_ben_snds,nb,total_nb,ratio
0,1,20,0,20006799,170874330,0.117085
1,1,20,20,17127019,170874330,0.100232
2,1,20,30,17833626,170874330,0.104367
3,1,20,40,18584273,170874330,0.108760
4,1,20,50,22067147,170874330,0.129143
...,...,...,...,...,...,...
1075,42,24,50,560449,3753047,0.149332
1076,42,24,60,653318,3753047,0.174077
1077,42,24,70,631355,3753047,0.168225
1078,42,24,80,352557,3753047,0.093939


In [4]:
sql =f"""
    select year, sum(pop) pop, sum(pop0002)/sum(pop) p0002, sum(pop0305)/sum(pop) p0305, sum(pop0610)/sum(pop) p0610, sum(pop1117)/sum(pop) p1117, sum(pop1824)/sum(pop) p1824, sum(pop2539)/sum(pop) p2539, sum(pop4054)/sum(pop) p4054, sum(pop5564)/sum(pop) p5564, sum(pop6579)/sum(pop) p6579, sum(pop80p)/sum(pop) p80p
    from iris.pop_iris
    group by year
"""
pop_iris = pd.read_sql(sql, config.connection_string)
pop_iris

,year,pop,p0002,p0305,p0610,p1117,p1824,p2539,p4054,p5564,p6579,p80p
0,20,67162154.0,0.032056,0.034700,0.061375,0.086881,0.080607,0.180414,0.195735,0.126311,0.140958,0.060963
1,21,67408052.0,0.031403,0.033951,0.060709,0.087117,0.081043,0.179629,0.194405,0.126668,0.144234,0.060842


In [5]:
pop_iris[pop_iris["year"]==20].iloc[0].to_dict()

{'year': 20.0,
 'pop': 67162153.99999999,
 'p0002': 0.03205633570254437,
 'p0305': 0.03469965383959878,
 'p0610': 0.06137455685564376,
 'p1117': 0.08688149205273836,
 'p1824': 0.08060697124120414,
 'p2539': 0.18041440053576852,
 'p4054': 0.19573528970808537,
 'p5564': 0.1263105848164089,
 'p6579': 0.14095808265358642,
 'p80p': 0.06096263259442076}

In [6]:
def compute_overrepresentation(damir_dataset, year, psp_spe_snds):
    yy=20 if year==20 else 21
    props=pop_iris[pop_iris["year"]==yy].iloc[0].to_dict()
    bb = 1 if psp_spe_snds==1 or psp_spe_snds==12 else 0
    damir = damir_dataset[damir_dataset["ann"]==year]
    damir = damir[damir["psp_spe_snds"]==psp_spe_snds]
    overs={"year":year, "psp_spe_snds":psp_spe_snds}
    overs["p0002"]=(damir[damir["age_ben_snds"]==0]["ratio"]*(3+bb*3)/20/(props["p0002"])).iloc[0]
    overs["p0305"]=(damir[damir["age_ben_snds"]==0]["ratio"]*3/20/(props["p0305"])).iloc[0]
    overs["p0610"]=(damir[damir["age_ben_snds"]==0]["ratio"]*(5-bb)/20/(props["p0610"])).iloc[0]
    overs["p1117"]=(damir[damir["age_ben_snds"]==0]["ratio"]*(7-bb)/20/(props["p1117"])).iloc[0]
    overs["p1824"]=((damir[damir["age_ben_snds"]==0]["ratio"]*(2-bb)/20+damir[damir["age_ben_snds"]==20]["ratio"].iloc[0]*5/10)/(props["p1824"])).iloc[0]
    overs["p2539"]=((damir[damir["age_ben_snds"]==20]["ratio"]*6/10+damir[damir["age_ben_snds"]==30]["ratio"].iloc[0]*10/10)/(props["p2539"])).iloc[0]
    overs["p4054"]=((damir[damir["age_ben_snds"]==40]["ratio"]*10/10+damir[damir["age_ben_snds"]==50]["ratio"].iloc[0]*5/10)/(props["p4054"])).iloc[0]
    overs["p5564"]=((damir[damir["age_ben_snds"]==50]["ratio"]*5/10+damir[damir["age_ben_snds"]==60]["ratio"].iloc[0]*5/10)/(props["p5564"])).iloc[0]
    overs["p6579"]=((damir[damir["age_ben_snds"]==60]["ratio"]*5/10+damir[damir["age_ben_snds"]==70]["ratio"].iloc[0]*10/10)/(props["p6579"])).iloc[0]
    overs["p80p"]=((damir[damir["age_ben_snds"]==80]["ratio"]*10/10+damir[damir["age_ben_snds"]==99]["ratio"].iloc[0]*10/10)/(props["p80p"])).iloc[0]
    if psp_spe_snds == 12: #Pédiatre
        overs["p0002"]+=overs["p1117"]
        overs["p0002"]+=overs["p1824"]
        overs["p0002"]+=overs["p2539"]
        overs["p0002"]+=overs["p4054"]
        overs["p0002"]+=overs["p5564"]
        overs["p0002"]+=overs["p6579"]
        overs["p0002"]+=overs["p80p"]
        overs["p1117"]=overs["p1824"]=overs["p2539"]=overs["p4054"]=overs["p5564"]=overs["p6579"]=overs["p80p"]=0
    return overs
    
overs = compute_overrepresentation(damir, 20, 7)
overs

{'year': 20,
 'psp_spe_snds': 7,
 'p0002': 0.39353802332982507,
 'p0305': 0.36355944776544424,
 'p0610': 0.34257917378935693,
 'p1117': 0.3388052193342174,
 'p1824': 1.3605532312341913,
 'p2539': 1.9373190933705244,
 'p4054': 1.0312562284721365,
 'p5564': 0.8286150729338242,
 'p6579': 0.9388933441230419,
 'p80p': 0.76290351609114}

In [6]:
def df_append_dict(df, dico):
    dico_df = pd.DataFrame([dico])
    return pd.concat([df, dico_df], ignore_index=True)

df = pd.DataFrame(columns=overs.keys())
df = df_append_dict(df, overs)
df

,year,psp_spe_snds,p0002,p0305,p0610,p1117,p1824,p2539,p4054,p5564,p6579,p80p
0,20,1,1.095742,0.506136,0.381542,0.404292,0.694358,0.911823,0.885539,1.09036,1.53239,2.481171


In [7]:
years=damir["ann"].unique()
print(years)
psp_spe_sndss=damir["psp_spe_snds"].unique()
psp_spe_sndss

[20 21 22 23 24]


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 12, 13, 14, 15, 18, 31, 32,
       33, 34, 35, 36, 37, 38, 42], dtype=int64)

In [8]:
df = pd.DataFrame(columns=overs.keys())
for year in years:
    for i in psp_spe_sndss:
        overs = compute_overrepresentation(damir, year, i)
        df = df_append_dict(df, overs)
df

,year,psp_spe_snds,p0002,p0305,p0610,p1117,p1824,p2539,p4054,p5564,p6579,p80p
0,20,1,1.095742,0.506136,0.381542,0.404292,0.694358,0.911823,0.885539,1.090360,1.532390,2.481171
1,20,2,0.469924,0.434127,0.409074,0.404568,0.710009,0.944169,0.972663,1.212574,1.687848,1.804696
2,20,3,0.182562,0.168655,0.158922,0.157172,0.322459,0.482571,0.836800,1.367936,2.310819,3.181369
3,20,4,0.378437,0.349608,0.329433,0.325804,0.650583,0.850601,0.994057,1.305577,1.793892,1.932939
4,20,5,0.668506,0.617582,0.581942,0.575531,0.986708,1.059836,0.871649,1.027769,1.450694,1.784825
...,...,...,...,...,...,...,...,...,...,...,...,...
115,24,35,0.180555,0.167004,0.155658,0.151862,0.360191,0.602321,0.888491,1.280214,2.195516,2.993326
116,24,36,3.216437,2.975034,2.772919,2.705292,1.323880,0.620283,0.404405,0.331865,0.336400,0.231363
117,24,37,0.225429,0.208510,0.194344,0.189605,0.780912,1.180008,1.135108,1.261153,1.533236,1.519102
118,24,38,0.450397,0.416593,0.388291,0.378821,0.697267,1.081763,0.946691,1.129341,1.590872,1.925953


In [9]:
df.to_sql("overrepresentation", config.connection_string, schema="apl", if_exists="replace", index=False)

120